In [ ]:
%pip install pandas

In [ ]:
import pandas as pd

# **Extração de Dados**

In [ ]:
# Dicionário com os caminhos para os arquivos.
arquivos = {
    '2021': 'data/recife-dados-receitas-2021.csv',
    '2022': 'data/recife-dados-receitas-2022.csv',
    '2023': 'data/recife-dados-receitas-2023.csv'
}

lista_dataframes = [] # Uma lista vazia para guardar os dados de cada ano temporariamente.

for ano, arquivo in arquivos.items():
    try:
        df_temp = pd.read_csv(
            arquivo,
            sep=';',           # Separador: CSVs do governo brasileiro usam ponto e vírgula.
            encoding='utf-8',  # Codificação: Para ler acentos (ç, ã, é) corretamente.
            thousands='.',     # Milhar: Avisa que o ponto serve para milhar (1.000).
            decimal=',',       # Decimal: Avisa que a vírgula é decimal (0,50).
            dtype=str          # Força a ler como texto inicialmente
        )
       
        # Cria uma coluna nova para saber de qual ano veio aquele dado.
        df_temp['ano'] = ano

        # Adicionando esse pedaço na nossa lista.
        lista_dataframes.append(df_temp)

    except Exception as e:
        # Se der erro (arquivo não existe, nome errado), recebemos uma mensagem de erro.
        print(f"Erro em {ano}: {e}")

## **Consolidação/Concatenação**

In [ ]:
# Só rodamos se a lista não estiver vazia.
if lista_dataframes:
    # pd.concat = Concatenar.
    df_raw = pd.concat(lista_dataframes, ignore_index=True)
    print(f"Total de linhas no df unificado: {len(df_raw)}")

Total de linhas no df unificado: 16621


## **Resultado e Informações**

In [ ]:
# Verificando o resultado final e as informações sobre os tipos de dados e 
display(df_raw.head(5))
display(df_raw.info())

,ano,mes,mes_nome,orgao_codigo,orgao_nome,unidade_codigo,unidade_nome,fonte_recurso_codigo,fonte_recurso_nome,categoria_receita_codigo,...,alinea_receita_codigo,alinea_receita_nome,subalinea_receita_codigo,subalinea_receita_nome,receita_local_codigo,receita_local_nome,receita_prevista,receita_prevista_acrescimo,receita_prevista_atualizada,receita_arrecadada
0,2021,1,Janeiro,15,SECRETARIA DE FINANÇAS,15.01,SECRETARIA DE FINANÇAS - ADMINISTRAÇÃO DIRETA,0100,RECURSOS ORDINÁRIOS - NÃO VINCULADOS,1,...,00,DIVIDENDOS,11,DIVIDENDOS - PRINCIPAL,0000,SEM PREVISÃO INICIAL PARA ESSA CLASSIFICAÇÃO,"1065,00","0,00","1065,00","0,00"
1,2021,1,Janeiro,15,SECRETARIA DE FINANÇAS,15.01,SECRETARIA DE FINANÇAS - ADMINISTRAÇÃO DIRETA,0100,RECURSOS ORDINÁRIOS - NÃO VINCULADOS,1,...,01,PARTICIPAÇÃO NA RECEITA DA UNIÃO,41,COTA-PARTE DO FUNDO DE PARTICIPAÇÃO DOS MUNICÍ...,0000,SEM PREVISÃO INICIAL PARA ESSA CLASSIFICAÇÃO,"22550000,00","0,00","22550000,00","0,00"
2,2021,1,Janeiro,15,SECRETARIA DE FINANÇAS,15.01,SECRETARIA DE FINANÇAS - ADMINISTRAÇÃO DIRETA,0100,RECURSOS ORDINÁRIOS - NÃO VINCULADOS,1,...,01,PARTICIPAÇÃO NA RECEITA DA UNIÃO,59,COTA-PARTE DO IMPOSTO SOBRE A PROPRIEDADE TERR...,0000,SEM PREVISÃO INICIAL PARA ESSA CLASSIFICAÇÃO,"-1300,00","0,00","-1300,00","0,00"
3,2021,1,Janeiro,15,SECRETARIA DE FINANÇAS,15.01,SECRETARIA DE FINANÇAS - ADMINISTRAÇÃO DIRETA,0100,RECURSOS ORDINÁRIOS - NÃO VINCULADOS,1,...,01,PARTICIPAÇÃO NA RECEITA DA UNIÃO,51,COTA-PARTE DO IMPOSTO SOBRE A PROPRIEDADE TERR...,0000,SEM PREVISÃO INICIAL PARA ESSA CLASSIFICAÇÃO,"6500,00","0,00","6500,00","0,00"
4,2021,1,Janeiro,15,SECRETARIA DE FINANÇAS,15.01,SECRETARIA DE FINANÇAS - ADMINISTRAÇÃO DIRETA,0100,RECURSOS ORDINÁRIOS - NÃO VINCULADOS,1,...,01,PARTICIPAÇÃO NA RECEITA DOS ESTADOS,31,COTA-PARTE DO IPI - MUNICÍPIOS - PRINCIPAL,0000,SEM PREVISÃO INICIAL PARA ESSA CLASSIFICAÇÃO,"2095000,00","0,00","2095000,00","0,00"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16621 entries, 0 to 16620
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   ano                          16621 non-null  object
 1   mes                          16621 non-null  object
 2   mes_nome                     16621 non-null  object
 3   orgao_codigo                 16621 non-null  object
 4   orgao_nome                   16621 non-null  object
 5   unidade_codigo               16621 non-null  object
 6   unidade_nome                 16621 non-null  object
 7   fonte_recurso_codigo         16621 non-null  object
 8   fonte_recurso_nome           16621 non-null  object
 9   categoria_receita_codigo     16621 non-null  object
 10  categoria_receita_nome       16621 non-null  object
 11  fonte_origem_receita_codigo  16621 non-null  object
 12  fonte_origem_receita_nome    16621 non-null  object
 13  subfonte_receita_codigo      16

None

## **Lógica de Limpeza**

In [ ]:
def limpar_moeda(valor):
        if isinstance(valor, str): # Só limpa se for texto
            valor_limpo = valor.replace('R$', '').strip()
            valor_limpo = valor_limpo.replace('.', '')    # Tira milhar
            valor_limpo = valor_limpo.replace(',', '.')   # Vírgula vira ponto
            
            try:
                return float(valor_limpo)
            except:
                return 0.0
        return valor

## **Aplicando a limpeza nas colunas de valor**

In [ ]:
cols_valores = ['receita_prevista', 'receita_prevista_acrescimo', 'receita_prevista_atualizada', 'receita_arrecadada']
for col in cols_valores:
    df_raw[col] = df_raw[col].apply(limpar_moeda)

## **Análise e Tratamento de Nulos**

In [ ]:
print("--- ANÁLISE DE VALORES NULOS POR COLUNA ---")

# Vamos mostrar a contagem de nulos por coluna
print(df_raw.isnull().sum())

# Agora, vamos verificar as 5 primeiras linhas do DataFrame que possuem a coluna 'rubrica_receita_nome' nula
coluna_teste = 'rubrica_receita_nome'

if coluna_teste in df_raw.columns:
    print(f"Mostrando as primeiras 5 linhas onde '{coluna_teste}' está nula:")
    
    # O comando .isnull() retorna um boolean. Usamos isso para filtrar.
    linhas_com_problema = df_raw[df_raw[coluna_teste].isnull()]
    
    if not linhas_com_problema.empty:
        display(linhas_com_problema.head())
    else:
        print(f"Nenhum valor nulo encontrado na coluna {coluna_teste}!")

coluna_categoria = 'categoria_receita_nome'
if coluna_categoria in df_raw.columns:
    print(f"Valores únicos em {coluna_categoria} (Amostra):")
    print(df_raw[coluna_categoria].unique()[:20]) # Mostra os 20 primeiros tipos encontrados

--- ANÁLISE DE VALORES NULOS POR COLUNA ---
ano                            0
mes                            0
mes_nome                       0
orgao_codigo                   0
orgao_nome                     0
unidade_codigo                 0
unidade_nome                   0
fonte_recurso_codigo           0
fonte_recurso_nome             0
categoria_receita_codigo       0
categoria_receita_nome         0
fonte_origem_receita_codigo    0
fonte_origem_receita_nome      0
subfonte_receita_codigo        0
subfonte_receita_nome          0
rubrica_receita_codigo         0
rubrica_receita_nome           0
alinea_receita_codigo          0
alinea_receita_nome            0
subalinea_receita_codigo       0
subalinea_receita_nome         0
receita_local_codigo           0
receita_local_nome             0
receita_prevista               0
receita_prevista_acrescimo     0
receita_prevista_atualizada    0
receita_arrecadada             0
dtype: int64
Mostrando as primeiras 5 linhas onde 'rubrica_receit

## **Carregamento dos dados: transformação em esquema estrela e injetando o dataset no PostgreSQL**
Para nosso projeto, decidimos organizar os dados nas seguintes tabelas dimensões:

- Tempo (ano,mês)
- Entidade (orgao_nome, unidade_nome)
- Fonte (Fonte_receita, subfonte_receita, fonte_recurso)
- Descrição (categoria, rubrica, alinea, subalinea, receita_local)
- Valores (receita_prevista, receita_acrescimo, receita_arrecadada)

In [ ]:
#1 Dimensão tempo (quando ocorreu)

colunas_tempo = ['ano', 'mes'] #separar as colunas necessárias para essa dimensão

dim_tempo = df_raw[colunas_tempo].drop_duplicates().reset_index(drop=True) #adicionar as colunas removendo as duplicatas
dim_tempo = dim_tempo.reset_index().rename(columns = {'index' : 'id_tempo_sk'}) # renomear a coluna index 

print(f"tabela dimensão tempo criada: {len(dim_tempo)} entradas únicas")

#2 Dimensão entidade (quem está envolvido)

colunas_entidade = ['orgao_nome', 'unidade_nome'] #separar as colunas necessárias para essa dimensão

dim_tempo = df_raw[colunas_entidade].drop_duplicates().reset_index(drop=True) #adicionar as colunas removendo as duplicatas
dim_tempo = dim_tempo.reset_index().rename(columns = {'index' : 'id_entidade_sk'}) # renomear a coluna index 

print(f"tabela dimensão entidade criada: {len(dim_tempo)} entradas únicas")

tabela dimensão tempo criada: 36 entradas únicas
tabela dimensão entidade criada: 32 entradas únicas


## Excluindo Colunas

Excluindo as colunas ~~nome_mes~~, ~~alinea~~, ~~subalinea~~ e ~~fonte~~, junto com seus codigos.

A coluna **_nome-mes_** está sendo redundante, uma vez que já existe uma coluna identificando o mês da receita.

As colunas **_alinea_** e **_subalinea_** são especificações que nem sempre adicionam informações, e, quando adicionam, na miaoria da vezes são informações fúteis.

A coluna **_fonte_** pode ser substituida pela coluna **_subfonte_** sem alterar as informações presentes, uma vez que ela (subfonte) é uma versão menos simplificada da fonte.


In [ ]:
remover = ['mes_nome', 'alinea_receita_codigo', 'alinea_receita_nome', 'subalinea_receita_codigo', 
           'subalinea_receita_nome', 'fonte_origem_receita_codigo', 'fonte_origem_receita_nome']

df_raw = df_raw.drop(columns=remover, axis=1)
display(df_raw.head())